In [18]:
import numpy as np
import pandas as pd
import time

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 30)
pd.set_option('display.width', 1000)


# Objective functions
def rastrigin(X):
    return 10 * len(X) + sum([(x ** 2 - 10 * np.cos(2 * np.pi * x)) for x in X])


def himmelblau(point):
    return (point[0] ** 2 + point[1] - 11) ** 2 + (point[0] + point[1] ** 2 - 7) ** 2


def three_hump_camel(point):
    return 2 * point[0] ** 2 - 1.05 * point[0] ** 4 + (point[0] ** 6 / 6) + point[0] * point[1] + point[1] ** 2

In [19]:
def genetic_algorithm_optimize(
        objective_function,
        domain,
        step_size,
        max_generations=1000,

        population_size=100,
        mutation_rate=0.1,
        crossover_rate=0.7,

        step_size_increase_factor=1.5,
        no_improve_limit=50,
        tolerance=0.001
):
    # Initialize the population
    dim = len(domain)
    population = np.random.uniform(domain[0], domain[1], (population_size, len(domain)))
    best_solution = np.random.uniform(domain[0], domain[1], dim)
    best_fitness = objective_function(best_solution)
    last_improvement = 0
    generation = 0

    for generation in range(max_generations):
        # Evaluate the population
        fitness = np.array([objective_function(individual) for individual in population])

        # Find the best individual
        current_best_value = np.min(fitness)
        current_best_solution = population[np.argmin(fitness)]

        # Update the best found solution
        if current_best_value < best_fitness:
            best_solution = current_best_solution
            best_fitness = current_best_value
            last_improvement = generation

        # Check for termination condition
        if best_fitness <= tolerance:
            return best_solution, best_fitness, generation, 'good'

        # Check if stuck in a local minimum
        if generation - last_improvement > no_improve_limit:  # No improvement in the last 50 generations
            return best_solution, best_fitness, generation, 'bad'

        # Selection - tournament selection
        selected_indices = np.random.randint(0, population_size, population_size)
        mating_pool = population[selected_indices]

        # Crossover - single point crossover
        offspring = []
        for i in range(0, population_size, 2):
            if np.random.rand() < crossover_rate:
                crossover_point = np.random.randint(1, len(domain))
                parent1 = mating_pool[i]
                parent2 = mating_pool[min(i + 1, population_size - 1)]
                child1 = np.concatenate([parent1[:crossover_point], parent2[crossover_point:]])
                child2 = np.concatenate([parent2[:crossover_point], parent1[crossover_point:]])
                offspring.append(child1)
                offspring.append(child2)
            else:
                offspring.append(mating_pool[i])
                offspring.append(mating_pool[min(i + 1, population_size - 1)])

        # Mutation
        offspring = np.array(offspring)
        mutations = np.random.rand(population_size, len(domain)) < mutation_rate
        mutation_amount = np.random.uniform(-step_size, step_size, (population_size, len(domain)))
        offspring[mutations] += mutation_amount[mutations]
        offspring = np.clip(offspring, domain[0], domain[1])  # Ensure offspring stay within domain

        # Replace the old population with the new one
        population = offspring

    # If the loop finishes without breaking early
    return best_solution, best_fitness, generation, 'bad'

In [47]:
def stochastic_local_search(
        objective_function,
        domain,
        step_size=0.1,
        max_generations=1000,

        step_size_increase_factor=1.5,
        no_improve_limit=50,
        tolerance=0.001
):
    dim = len(domain)
    best_solution = np.random.uniform(domain[0], domain[1], dim)
    best_fitness = objective_function(best_solution)
    last_improvement = 0
    generation = 0

    for generation in range(max_generations):
        improved = False
        candidate_solution = best_solution + np.random.uniform(-step_size, step_size, dim)
        candidate_solution = np.clip(candidate_solution, domain[0], domain[1])
        candidate_value = objective_function(candidate_solution)

        if candidate_value < best_fitness:
            best_solution = candidate_solution
            best_fitness = candidate_value
            last_improvement = generation
            improved = True

        if improved:
            last_improvement = generation
            step_size *= step_size_increase_factor
        else:
            # Decrease step size if no improvement
            step_size /= step_size_increase_factor

        # Check for good solution or if the search is stuck
        if best_fitness <= tolerance:
            return best_solution, best_fitness, generation, 'good'

        if generation - last_improvement > no_improve_limit:
            return best_solution, best_fitness, generation, 'bad'

    # Return the final state if no other return has been triggered
    return best_solution, best_fitness, generation, 'bad' if generation - last_improvement > no_improve_limit else 'good'

In [48]:
def deterministic_local_search(
        objective_function,
        domain,
        step_size=0.1,
        max_generations=1000,

        step_size_increase_factor=1.5,
        no_improve_limit=50,
        tolerance=0.001
):
    dim = len(domain)
    best_solution = np.random.uniform(domain[0], domain[1], dim)
    best_fitness = objective_function(best_solution)
    last_improvement = 0
    generation = 0

    for generation in range(max_generations):
        improved = False
        for i in range(dim):
            for direction in [-1, 1]:
                neighbor = np.copy(best_solution)
                neighbor[i] += direction * step_size
                neighbor = np.clip(neighbor, domain[0], domain[1])
                neighbor_value = objective_function(neighbor)

                # Check if the neighbor is better
                if neighbor_value < best_fitness:
                    best_solution = neighbor
                    best_fitness = neighbor_value
                    last_improvement = generation
                    improved = True

        if improved:
            last_improvement = generation
            step_size *= step_size_increase_factor
        else:
            # Decrease step size if no improvement
            step_size /= step_size_increase_factor

        if best_fitness <= tolerance:
            return best_solution, best_fitness, generation, 'good'

        # Check if stuck in a local minimum
        if generation - last_improvement > no_improve_limit:  # No improvement in the last 50 generations
            return best_solution, best_fitness, generation, 'bad'

    return best_solution, best_fitness, generation, 'bad'

In [49]:
DOMAIN_BOUND = 5
STEP_SIZE_BOUND = 1


def run_tests():
    results1 = []
    results2 = []
    results3 = []

    test_functions = [rastrigin, himmelblau, three_hump_camel]

    for func in test_functions:
        for i in range(100):
            domain = np.random.uniform(-DOMAIN_BOUND, 0, 1).tolist() + np.random.uniform(0, DOMAIN_BOUND, 1).tolist()
            step_size = np.random.choice(np.arange(0.1, (STEP_SIZE_BOUND + .1), 0.1))

            start_time = time.time()
            best_solution, best_value, iteration, result = genetic_algorithm_optimize(func, domain=domain,
                                                                                      step_size=step_size)
            end_time = time.time()
            results1.append({
                'Function': func.__name__,
                'Best Solution': [round(x, 4) for x in best_solution],
                'Best Fitness': round(best_value, 5),
                'Execution Time': round(end_time - start_time, 4),
                'Step Size': round(step_size, 4),
                'Domain': [round(x, 4) for x in domain],
                'Iterations': iteration,
                'Result': result
            })

            start_time = time.time()
            best_solution, best_value, iteration, result = deterministic_local_search(func, domain=domain,
                                                                                      step_size=step_size)
            end_time = time.time()
            results2.append({
                'Function': func.__name__,
                'Best Solution': [round(x, 4) for x in best_solution],
                'Best Fitness': round(best_value, 5),
                'Execution Time': round(end_time - start_time, 4),
                'Step Size': round(step_size, 4),
                'Domain': [round(x, 4) for x in domain],
                'Iterations': iteration,
                'Result': result
            })

            start_time = time.time()
            best_solution, best_value, iteration, result = stochastic_local_search(func, domain=domain,
                                                                                   step_size=step_size, tolerance=0.01)
            end_time = time.time()
            results3.append({
                'Function': func.__name__,
                'Best Solution': [round(x, 4) for x in best_solution],
                'Best Fitness': round(best_value, 5),
                'Execution Time': round(end_time - start_time, 4),
                'Step Size': round(step_size, 4),
                'Domain': [round(x, 4) for x in domain],
                'Iterations': iteration,
                'Result': result
            })

    df_results1 = pd.DataFrame(results1)
    df_results2 = pd.DataFrame(results2)
    df_results3 = pd.DataFrame(results3)

    return df_results1, df_results2, df_results3

In [50]:
df1, df2, df3 = run_tests()

In [54]:
df1.to_csv("./genetic_algorithm_optimize.csv", index=False)
df2.to_csv("./deterministic_local_search.csv", index=False)
df3.to_csv("./stochastic_local_search.csv", index=False)

In [55]:
def process_df(df):
    # Create quantiles for Domain and Step Size
    df['Domain_left'] = df['Domain'].apply(lambda x: x[0])
    df['Domain_right'] = df['Domain'].apply(lambda x: x[1])

    # Flatten the Domain quantiles into a single range
    df['Domain_range'] = df['Domain_right'] - df['Domain_left']

    # Create quantiles
    df['Domain_quantile'] = pd.qcut(df['Domain_range'], q=15, labels=False, duplicates='drop')
    df['Step_Size_quantile'] = pd.qcut(df['Step Size'], q=15, labels=False, duplicates='drop')

    # Filter data
    filtered_data = df[
            (df['Domain_quantile'] < 15) &
            (df["Step_Size_quantile"] < 15)
        ]

    # Group and aggregate data
    grouped_data = filtered_data.groupby(['Result', 'Function']).agg({
        'Best Fitness': 'mean',
        'Execution Time': 'mean',
        'Iterations': 'mean',
        'Step Size': 'mean',
        'Domain_range': 'mean',
        'Result': 'count',
    }).rename(columns={
        'Best Fitness': 'Avg Fitness',
        'Execution Time': 'Avg Exe. Time',
        'Step Size': 'Avg Step Size',
        'Iterations': 'Avg Iter',
        'Domain_range': 'Avg Domain',
        'Result': 'Count',
    })

    # Merge counts with grouped data
    return grouped_data

In [56]:
results = [
    {"df": process_df(df1), "method": "Genetic Algorithm Optimize"},
    {"df": process_df(df2), "method": "Deterministic Local Search"},
    {"df": process_df(df3), "method": "Stochastic Local Search"}]

for i, result in enumerate(results, start=1):
    print(f"Results for DataFrame {result['method']}:")
    print(result['df'])
    print()

Results for DataFrame Genetic Algorithm Optimize:
                         Avg Fitness  Avg Exe. Time   Avg Iter  Avg Step Size  Avg Domain  Count
Result Function                                                                                 
bad    himmelblau          22.274086       0.033603  70.806122       0.533673    5.085559     98
       rastrigin            0.807389       0.063738  76.060606       0.563636    5.091675     99
       three_hump_camel     0.017182       0.036386  70.986111       0.551389    5.697453     72
good   himmelblau           0.000495       0.036750  78.000000       0.450000    5.412950      2
       rastrigin            0.000890       0.115800  56.000000       0.600000    0.863900      1
       three_hump_camel     0.000572       0.015775  30.571429       0.550000    3.679489     28

Results for DataFrame Deterministic Local Search:
                         Avg Fitness  Avg Exe. Time    Avg Iter  Avg Step Size  Avg Domain  Count
Result Function          